# Andrew Ng Coursera Machine Learning Course - Ex 2
**Dean's Reimplementation Attempt**

*9/23/2017*

Note: See [here](https://docs.scipy.org/doc/scipy/reference/api.html#guidelines-for-importing-functions-from-scipy) for recommendations on how to import `scipy` modules.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
%matplotlib inline

## 1. Logistic Regression
**Note:** This is a classification algorithm, despite the term "regression" in the name.  Classification means outputs $y$ take on discrete values, for example $y \in \{0,1\}$.  The output is  also known as a *label*.

*Sigmoid function*, aka *logistic function*

$$ g(z) = \frac{1}{1 + e^{-z}}$$

In [ ]:
z = np.linspace(-4, 4, 81)
g = 1 / (1 + np.exp(-z))
plt.figure(figsize=(10, 1.5))
plt.plot(z, g)

For the hypothesis, use:

$$z = \theta^Tx$$

$$h_\theta(x) = g(z) = g(\theta^Tx) = \frac{1}{1 + e^{-\theta^Tx}} $$

... making $h_\theta(x)$ the probability that the output is 1, not 0.

$$h_\theta(x) = P(y=1 \: | \: x;\theta) $$

For the decision boundary, if $h_\theta(x) \ge 0.5$, predict $y = 1$.  This happens when $z \ge 0$.

For the cost function, use:

$$\text{Cost}(h_\theta(x), y) = - \log_e(h_\theta(x)) \qquad\qquad \text{if} \, y=1 $$
$$\text{Cost}(h_\theta(x), y) = - \log_e(1 - h_\theta(x)) \qquad \text{if} \, y=0 $$
$$J(\theta) = \frac{1}{m} \sum_{i=1}^m \text{Cost}\left(h_\theta(x^{(i)}, y^{(i)}\right) $$

...so...

$$J(\theta) = -\frac{1}{m} \sum_{i=1}^m \left[ 
y^{(i)} \log(h_\theta(x^{(i)}))
+ (1-y^{(i)}) \log(1 - h_\theta(x^{(i)}))
\right] $$

Vectorized, this is:
$$J(\theta) = \frac{1}{m} \left( -y^\top\log(h) - (1-y)^\top\log(1-h) \right) $$

The gradient is:
$$\frac{\partial}{\partial\theta_j} = \frac{1}{m} \sum_{i=1}^m
  \left( h_\theta(x^{(i)}) - y^{(i)} \right) x_j^{(i)}$$

So the gradient descent rule is:

$$\theta_j := \theta_j - \frac{\alpha}{m} \sum_{i=1}^m
  \left( h_\theta(x^{(i)}) - y^{(i)} \right) x_j^{(i)}$$
...which *looks* the same as for linear regression, but remember that $h_\theta(x^{(i)})$ is the sigmoid function this time.

## 1.1 Visualizing the data

See [numpy.matmul() documentation](https://docs.scipy.org/doc/numpy/reference/generated/numpy.matmul.html) for how a 1-D numpy array is treated by matrix multiplication.  

Basically, given vector v and matrix M, in **`v@M`** the v is treated as a row vector, but in **`M@v`** the v is treated as a column vector.

In [ ]:
data = np.loadtxt('ex2/ex2data1.txt', delimiter=',')
X = data[:, 0:2]
y = data[:, 2]    # y = data[:, 2:3] for 1-column 2-d array.
print(data.shape)
print(X.shape)
print(y.shape)
print(data[:5])
print(X[:5])
print(y[:5])

In [ ]:
pos = np.where(y == 1)
neg = np.where(y == 0)

In [ ]:
# Save lists of "lines" created (l1...) for the legend below.
l1 = plt.plot(X[pos, 0], X[pos, 1], 'k+', label='Admitted')
l2 = plt.plot(X[neg, 0], X[neg, 1], 'yo', label='Not admitted')

# plt.legend() puts one legend entry for each data point,
# so pick just the first point from each class and specify those.
plt.legend((l1[0], l2[0]), (l1[0].get_label(), l2[0].get_label()))

plt.xlabel('Exam 1 score')
plt.ylabel('Exam 2 score')
plt.show()

# 1.2. Implementation

In [ ]:
# Add the intercept/bias term of all 1's to X.
m = X.shape[0]
ones = np.ones((m, 1))
X = np.hstack((ones, X))
print(X[:5])

## 1.2.1. Warmup exercise: sigmoid function
*Sigmoid function*, aka *logistic function*

$$ g(z) = \frac{1}{1 + e^{-z}}$$

In [ ]:
# Student implements...
def sigmoid(X):
    g = 1 / (1 + np.exp(-X))
    return g

## 1.2.2. Cost function and gradient

$$J(\theta) = -\frac{1}{m} \sum_{i=1}^m \left[ 
y^{(i)} \log(h_\theta(x^{(i)}))
+ (1-y^{(i)}) \log(1 - h_\theta(x^{(i)}))
\right] $$

$$\frac{\partial}{\partial\theta_j} = \frac{1}{m} \sum_{i=1}^m
  \left( h_\theta(x^{(i)}) - y^{(i)} \right) x_j^{(i)}$$

In [ ]:
initial_theta = np.zeros(X.shape[1]) # np.zeros((X.shape[1], 1)), for n x 1 2d array
initial_theta

In [ ]:
# Student implements...
def costFunction(theta, X, y):
    m = X.shape[0]
    h = sigmoid(X @ theta) # Not theta.T @ X because I wanted samples in rows?

    Jsumofones = y.T @ np.log(h) 
    Jsumofzeros = (1-y).T @ np.log(1-h)
    J = (-1/m) * (Jsumofones + Jsumofzeros)
    
    grad = (1/m) * X.T @ (h - y)

    return J, grad

In [ ]:
cost, grad = costFunction(initial_theta, X, y)

In [ ]:
# Expected 0.693, and [-0.1, -12.0092, -11.2628]
print(cost) # 1-d vector input made this a scalar, not a 2-d array.
print(grad) # 1-d vector input etc made this a 1-d array, not a 2-d array.

In [ ]:
# Extra parentheses and commas would make the following a column vector.
#test_theta = np.array(((-24,), (0.2,), (0.2,)))
test_theta = np.array((-24, 0.2, 0.2))
print(test_theta.shape)
cost, grad = costFunction(test_theta, X, y)

In [ ]:
# Expected cost (approx): 0.218
# Expected gradients (approx):  0.043, 2.566, 2.647');
print(cost) # Should this be coming out as a scalar, not a 2-d array?
print(grad) # Should this be coming out as a 1-d array, not a 2-d array?

### 1.2.3. Learning parameters using `scipy.optimize.minimize`
`scipy.optimize.minimize()` calls back to the objective function with only the parameter vector being optimized (in our case, `theta`).  So we need to make a partial function that only accepts `theta` and always uses X and y built-in.  We can do this as separate function definition or as a lambda.  So first we make basically a partial function based on our `costFunction()` that will still accept `theta`, but will fill in `X` and `y` for us.

Note: This has to do with partial as a functional programming concept, not partial derivatives.

Note2: It would have been nice to do this with the `partial()` function in the `functools` module, but I don't think the original order of parameters allows that.

Note3: Whichever way we pick, this is the same thing as Andrew Ng's original does in Octave/Matlab with the `@(t) (costFunction(t, X, y))` syntax. `optimize.minimize(lambda t: costFunction(t, X, y), ...

In [ ]:
# The non-lambda way to make a cost function to pass to optimize.minimize()
#def costFun(theta):
#    J, grad = costFunction(theta, X, y)
#    return J, grad
#    # return J[0], grad[:,0] # This was if using 2-d theta input.

In [ ]:
initial_theta = np.zeros([X.shape[1]])
initial_theta = test_theta

Note:  Was using `fminunc()` in Matlab

In [ ]:
%%time
result = optimize.minimize(
    lambda t: costFunction(t, X, y), 
    initial_theta, 
    jac=True, 
    options={'maxiter':100, 'disp':True}
    )

In [ ]:
result

In [ ]:
# Print theta to screen
print('Cost at theta found by optimize: %s' % result.fun)
print('Expected cost (approx):          0.203')
print('theta:                           %s' % result.x)
print('Expected theta (approx):         -25.161, 0.206, 0.201')

### 1.2.4. Evaluating Logistic Regression
TODO: Graph decision boundary.

Try predicting one student.

In [ ]:
theta = result.x
#X_one = np.array(((1, 45, 85), ))
X_one = np.array((1, 45, 85))
h_one = sigmoid(X_one @ theta)
h_one

In [ ]:
# Hmmm.
print('For scores 45 and 85, we predict admission probability %f' % h_one)
print('Expected value: 0.775 +/- 0.002')

Now check overall accuracy on the data we trained on.

In [ ]:
# Student implements...
def predict(theta, X):
    probabilities = sigmoid(X @ theta)
    return (probabilities >= 0.5).astype(np.float)
#predict(theta, X)

In [ ]:
p = predict(theta, X)
accuracy = (p == y).astype(np.float).mean()
accuracy

print('Train Accuracy: %f%%', accuracy * 100);
print('Expected accuracy (approx): 89.0');

# 2. Regularized Logistic Regression

In [ ]:
data = np.loadtxt('ex2/ex2data2.txt', delimiter=',')
X = data[:, 0:-1]
y = data[:, -1]
print(data[:5])
print(X[:5])
print(y[:5])

In [ ]:
pos = np.where(y == 1)
neg = np.where(y == 0)

## 2.1. Visualizing the data

** Why does this plot look different than Ex2.pdf handout? **

In [ ]:
l1 = plt.plot(X[pos, 0], X[pos, 1], 'k+', label='Passed')
l2 = plt.plot(X[neg, 0], X[neg, 1], 'yo', label='Failed')
plt.legend((l1[0], l2[0]), (l1[0].get_label(), l2[0].get_label()))

## 2.2. Feature mapping

In addition to adding the bias term of 1's, we map the 2 features into a bunch of features that include higher order polynomials of those two features.  This lets us fit more complex functions, although it is more prone to overfitting as well.

In [ ]:
###
# Add leading 1's column for intercept/bias term, and add 
# polynomial terms, but only for two starting features x1 & x2.
def mapFeature(x1, x2, degree=6, bias=True):

    # Calculate total number of terms. (Is there a more general way?)
    cols = degree * (degree + 3) // 2
    if bias:
        cols += 1

    # Allocate uninitialized array
    out = np.empty((len(x1), cols))
    nextcol = 0
    
    # Add bias term if requested
    if bias:
        out[:, nextcol] = 1
        nextcol += 1

    for i in range(1, degree + 1):
        for j in range(0, i + 1):
            out[:, nextcol] = (x1 ** (i-j)) * (x2 ** j)
            nextcol += 1

    return out

In [ ]:
print(X.shape)
print(X[:2])
X = mapFeature(X[:,0], X[:,1])
print(X.shape)
print(X[:2])

# 2.3. Cost function and gradient

In [ ]:
# Initialize parameters and lambda
initial_theta = np.zeros(X.shape[1])
lamb = 1;

In [ ]:
# Student implements cost function with regularzation
def costFunctionReg(initial_theta, X, y, lamb):
    pass

In [ ]:
cost, grad = costFunctionReg(initial_theta, X, y, lamb)
print('Cost at initial theta (zeros): ', cost)
print('Expected cost (approx): 0.693')
print('Gradient at initial theta (zeros) - first five values only:')
print(grad[:5])
print('Expected gradients (approx) - first five values only:')
print('0.0085, 0.0188, 0.0001, 0.0503, 0.0115')
